The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:


# keep pip up to date
%pip install -U pip

# install d2l but skip its (too strict) dependencies
%pip install d2l==1.0.3 --no-deps

# install dependencies compatible with Python 3.12
# NumPy >= 1.26 has Py3.12 wheels
%pip install "numpy>=1.26,<2" matplotlib pandas jupyter

# Choose the right index for your runtime (CPU vs CUDA). Example for CUDA 12.4:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
# Or CPU-only:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

import d2l, numpy as np
print("d2l OK, numpy:", np.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 171.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 90.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 169.4 MB/s  0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [jupyter]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
d2l 1.0.3 requires jupyter==1.0.0, but you have jupyter 1.1.1 which is incompatible.
d2l 1.0.3 requires matplotlib==3.7.2, but you have matplotlib 3.10.0 which is incompat

Looking in indexes: https://download.pytorch.org/whl/cpu
d2l OK, numpy: 2.0.2


# GPUs
:label:`sec_use_gpu`

In :numref:`tab_intro_decade`, we illustrated the rapid growth
of computation over the past two decades.
In a nutshell, GPU performance has increased
by a factor of 1000 every decade since 2000.
This offers great opportunities but it also suggests
that there was significant demand for such performance.


In this section, we begin to discuss how to harness
this computational performance for your research.
First by using a single GPU and at a later point,
how to use multiple GPUs and multiple servers (with multiple GPUs).

Specifically, we will discuss how
to use a single NVIDIA GPU for calculations.
First, make sure you have at least one NVIDIA GPU installed.
Then, download the [NVIDIA driver and CUDA](https://developer.nvidia.com/cuda-downloads)
and follow the prompts to set the appropriate path.
Once these preparations are complete,
the `nvidia-smi` command can be used
to (**view the graphics card information**).


In PyTorch, every array has a device; we often refer it as a *context*.
So far, by default, all variables
and associated computation
have been assigned to the CPU.
Typically, other contexts might be various GPUs.
Things can get even hairier when
we deploy jobs across multiple servers.
By assigning arrays to contexts intelligently,
we can minimize the time spent
transferring data between devices.
For example, when training neural networks on a server with a GPU,
we typically prefer for the model's parameters to live on the GPU.


To run the programs in this section,
you need at least two GPUs.
Note that this might be extravagant for most desktop computers
but it is easily available in the cloud, e.g.,
by using the AWS EC2 multi-GPU instances.
Almost all other sections do *not* require multiple GPUs, but here we simply wish to illustrate data flow between different devices.


In [1]:
import torch
from torch import nn
from d2l import torch as d2l

## [**Computing Devices**]

We can specify devices, such as CPUs and GPUs,
for storage and calculation.
By default, tensors are created in the main memory
and then the CPU is used for calculations.


In PyTorch, the CPU and GPU can be indicated by `torch.device('cpu')` and `torch.device('cuda')`.
It should be noted that the `cpu` device
means all physical CPUs and memory.
This means that PyTorch's calculations
will try to use all CPU cores.
However, a `gpu` device only represents one card
and the corresponding memory.
If there are multiple GPUs, we use `torch.device(f'cuda:{i}')`
to represent the $i^\textrm{th}$ GPU ($i$ starts at 0).
Also, `gpu:0` and `gpu` are equivalent.


In [2]:
def cpu():  #@save
    """Get the CPU device."""
    return torch.device('cpu')

def gpu(i=0):  #@save
    """Get a GPU device."""
    return torch.device(f'cuda:{i}')

cpu(), gpu(), gpu(1)

(device(type='cpu'),
 device(type='cuda', index=0),
 device(type='cuda', index=1))

We can (**query the number of available GPUs.**)


In [3]:
def num_gpus():  #@save
    """Get the number of available GPUs."""
    return torch.cuda.device_count()

num_gpus()

1

Now we [**define two convenient functions that allow us
to run code even if the requested GPUs do not exist.**]


In [4]:
def try_gpu(i=0):  #@save
    """Return gpu(i) if exists, otherwise return cpu()."""
    if num_gpus() >= i + 1:
        return gpu(i)
    return cpu()

def try_all_gpus():  #@save
    """Return all available GPUs, or [cpu(),] if no GPU exists."""
    return [gpu(i) for i in range(num_gpus())]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## Tensors and GPUs


By default, tensors are created on the CPU.
We can [**query the device where the tensor is located.**]


In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

It is important to note that whenever we want
to operate on multiple terms,
they need to be on the same device.
For instance, if we sum two tensors,
we need to make sure that both arguments
live on the same device---otherwise the framework
would not know where to store the result
or even how to decide where to perform the computation.

### Storage on the GPU

There are several ways to [**store a tensor on the GPU.**]
For example, we can specify a storage device when creating a tensor.
Next, we create the tensor variable `X` on the first `gpu`.
The tensor created on a GPU only consumes the memory of this GPU.
We can use the `nvidia-smi` command to view GPU memory usage.
In general, we need to make sure that we do not create data that exceeds the GPU memory limit.


In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

Assuming that you have at least two GPUs, the following code will (**create a random tensor, `Y`, on the second GPU.**)


In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.3855, 0.9210, 0.9824],
        [0.5975, 0.6907, 0.1974]])

### Copying

[**If we want to compute `X + Y`,
we need to decide where to perform this operation.**]
For instance, as shown in :numref:`fig_copyto`,
we can transfer `X` to the second GPU
and perform the operation there.
*Do not* simply add `X` and `Y`,
since this will result in an exception.
The runtime engine would not know what to do:
it cannot find data on the same device and it fails.
Since `Y` lives on the second GPU,
we need to move `X` there before we can add the two.

![Copy data to perform an operation on the same device.](http://d2l.ai/_images/copyto.svg)
:label:`fig_copyto`


In [9]:
Z = X.cuda(1)
print(X)
print(Z)

AcceleratorError: CUDA error: invalid device ordinal
GPU device may be out of range, do you have enough GPUs?
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


Now that [**the data (both `Z` and `Y`) are on the same GPU), we can add them up.**]


In [10]:
Y + Z

NameError: name 'Z' is not defined

But what if your variable `Z` already lived on your second GPU?
What happens if we still call `Z.cuda(1)`?
It will return `Z` instead of making a copy and allocating new memory.


In [ ]:
Z.cuda(1) is Z

True

### Side Notes

People use GPUs to do machine learning
because they expect them to be fast.
But transferring variables between devices is slow: much slower than computation.
So we want you to be 100% certain
that you want to do something slow before we let you do it.
If the deep learning framework just did the copy automatically
without crashing then you might not realize
that you had written some slow code.

Transferring data is not only slow, it also makes parallelization a lot more difficult,
since we have to wait for data to be sent (or rather to be received)
before we can proceed with more operations.
This is why copy operations should be taken with great care.
As a rule of thumb, many small operations
are much worse than one big operation.
Moreover, several operations at a time
are much better than many single operations interspersed in the code
unless you know what you are doing.
This is the case since such operations can block if one device
has to wait for the other before it can do something else.
It is a bit like ordering your coffee in a queue
rather than pre-ordering it by phone
and finding out that it is ready when you are.

Last, when we print tensors or convert tensors to the NumPy format,
if the data is not in the main memory,
the framework will copy it to the main memory first,
resulting in additional transmission overhead.
Even worse, it is now subject to the dreaded global interpreter lock
that makes everything wait for Python to complete.


## [**Neural Networks and GPUs**]

Similarly, a neural network model can specify devices.
The following code puts the model parameters on the GPU.


In [11]:
net = nn.Sequential(nn.LazyLinear(1))
net = net.to(device=try_gpu())

We will see many more examples of
how to run models on GPUs in the following chapters,
simply because the models will become somewhat more computationally intensive.

For example, when the input is a tensor on the GPU, the model will calculate the result on the same GPU.


In [12]:
net(X)

tensor([[0.3610],
        [0.3610]], device='cuda:0', grad_fn=<AddmmBackward0>)

Let's (**confirm that the model parameters are stored on the same GPU.**)


In [13]:
net[0].weight.data.device

device(type='cuda', index=0)

Let the trainer support GPU.


In [14]:
@d2l.add_to_class(d2l.Trainer)  #@save
def __init__(self, max_epochs, num_gpus=0, gradient_clip_val=0):
    self.save_hyperparameters()
    self.gpus = [d2l.gpu(i) for i in range(min(num_gpus, d2l.num_gpus()))]

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_batch(self, batch):
    if self.gpus:
        batch = [a.to(self.gpus[0]) for a in batch]
    return batch

@d2l.add_to_class(d2l.Trainer)  #@save
def prepare_model(self, model):
    model.trainer = self
    model.board.xlim = [0, self.max_epochs]
    if self.gpus:
        model.to(self.gpus[0])
    self.model = model

In short, as long as all data and parameters are on the same device, we can learn models efficiently. In the following chapters we will see several such examples.

## Summary

We can specify devices for storage and calculation, such as the CPU or GPU.
  By default, data is created in the main memory
  and then uses the CPU for calculations.
The deep learning framework requires all input data for calculation
  to be on the same device,
  be it CPU or the same GPU.
You can lose significant performance by moving data without care.
  A typical mistake is as follows: computing the loss
  for every minibatch on the GPU and reporting it back
  to the user on the command line (or logging it in a NumPy `ndarray`)
  will trigger a global interpreter lock which stalls all GPUs.
  It is much better to allocate memory
  for logging inside the GPU and only move larger logs.

## Exercises

1. Try a larger computation task, such as the multiplication of large matrices,
   and see the difference in speed between the CPU and GPU.
   What about a task with a small number of calculations?
1. How should we read and write model parameters on the GPU?
1. Measure the time it takes to compute 1000
   matrix--matrix multiplications of $100 \times 100$ matrices
   and log the Frobenius norm of the output matrix one result at a time. Compare it with keeping a log on the GPU and transferring only the final result.
1. Measure how much time it takes to perform two matrix--matrix multiplications
   on two GPUs at the same time. Compare it with computing in in sequence
   on one GPU. Hint: you should see almost linear scaling.


[Discussions](https://discuss.d2l.ai/t/63)


In [48]:
import time

Z = torch.rand(100, 10000)
X = torch.rand(100, 10000)

start = time.time()
Y = Z * X
end = time.time()

print(f"CPU Time: {end - start:.6f} seconds")

CPU Time: 0.003137 seconds


In [49]:
device = torch.device("cuda")

Z = torch.rand(100, 10000, device=device)
X = torch.rand(100, 10000, device=device)

torch.cuda.synchronize() # ensure GPU idle before timing
start = time.time()
Y = Z * X

torch.cuda.synchronize() # ensure GPU operations finished before returning control to CPU
end = time.time()

print(f"GPU Time: {end - start:.6f} seconds")

GPU Time: 0.000215 seconds


With small number of calculations, we do not benefit from using GPU compared to using CPU for calculations.

2/ Model parameters should be saved and loaded using state_dict which serializes via CPU and disk. After loading from disk, the data can be used for calculations on GPU, direct loading is not supported because GPU memory is volatile, data loss when process finishes.

In [50]:
X = torch.ones(2, 4, device=try_gpu())
Y = torch.ones(3, 4) # on GPU

net = nn.Sequential(nn.LazyLinear(2)) # materialize : 2 out features
net = net.to(device=try_gpu())
net(X)

new_net = nn.Sequential(nn.LazyLinear(2)) # materialize: 2 out features
new_net(Y)

torch.save(new_net.state_dict(), "model_state.pt")


In [51]:
net(X)

tensor([[-0.7517,  0.0565],
        [-0.7517,  0.0565]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [52]:
new_net(Y)

tensor([[0.4990, 0.1967],
        [0.4990, 0.1967],
        [0.4990, 0.1967]], grad_fn=<AddmmBackward0>)

In [53]:
net.load_state_dict(torch.load("model_state.pt"))
net.to(device="cuda")

Sequential(
  (0): Linear(in_features=4, out_features=2, bias=True)
)

In [54]:
# Exercise 3
A = torch.rand(100, 100, device = 'cuda')
B = torch.rand(100, 100, device = 'cuda')
log = []

start = time.time()
for i in range(1000):
  C = A @ B
  log.append(torch.norm(C, p = 'fro').item())

end = time.time()
print(f"CPU Time: {end - start:.6f} seconds")

CPU Time: 0.060848 seconds


In [55]:
logs = torch.empty(1000, device = 'cuda')

start = time.time()
for i in range(1000):
  C = A @ B
  logs[i] = torch.norm(C, p = 'fro') # on GPU

final = logs.cpu()
end = time.time()
print(f"GPU Time: {end - start:.6f} seconds")

GPU Time: 0.059095 seconds


In [56]:
# Exercise 4
X = torch.rand(3, 2, device=try_gpu())
Y = torch.rand(2, 3, device=try_gpu())

torch.cuda.synchronize()
start = time.time()
Z = torch.matmul(X, Y)

torch.cuda.synchronize()
end = time.time()
print(f"GPU Time on same GPU: {end - start:.6f} seconds")

GPU Time on same GPU: 0.000873 seconds


In [57]:
X = torch.rand(3, 2, device=try_gpu())
Y = torch.rand(2, 3, device=try_gpu(1))
Z = X.cuda(1)

torch.cuda.synchronize()
start = time.time()
Z = torch.matmul(X, Y)

torch.cuda.synchronize()
end = time.time()
print(f"GPU Time on different GPU: {end - start:.6f} seconds") # Calculation will be slower because of copying tensor, sometimes overhead of inter-device data transfer and synchronization

AcceleratorError: CUDA error: invalid device ordinal
GPU device may be out of range, do you have enough GPUs?
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
